In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
df = pd.read_csv('data.csv', sep = '|')

In [3]:
df

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,Windows.Internal.Shell.Broker.dll,09e83f1d1c99ad33009dbe6fb129c2d9,34404,240,8226,14,12,779776,253952,0,...,2,3.072050,2.706833,3.437268,604.000000,200,1008,256,16,1
1,hidserv.dll,3030f19c6a73367d6d5eedd157f5d01a,34404,240,8226,14,12,21504,13312,0,...,3,3.056204,2.709457,3.465499,515.333333,200,932,256,16,1
2,DmApiSetExtImplDesktop.dll,8271846f8f5dd1cfddaa957b1b9da1a2,34404,240,8226,14,12,33792,27648,0,...,1,3.466474,3.466474,3.466474,980.000000,980,980,256,16,1
3,FSResizerSetup27.exe,5802b4215566bb9593a736e945a28e99,332,224,271,6,0,23552,164864,1024,...,9,3.032143,1.971335,5.102472,363.444444,34,744,0,0,1
4,asc-setup.exe,8cb1fb45489d065720285deeccbacd98,332,224,33167,2,25,87040,71680,0,...,19,4.402265,2.699131,6.600332,3275.473684,16,33512,0,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10534,VirusShare_dd619efac1cea076816815b2f6220a6d,dd619efac1cea076816815b2f6220a6d,332,224,271,6,0,40960,32768,0,...,6,3.322505,2.370859,3.906893,401.666667,34,848,0,20,0
10535,VirusShare_c93fb2f6ef3a92dc0a0d33fc3f8668a9,c93fb2f6ef3a92dc0a0d33fc3f8668a9,332,224,258,14,11,286720,1044992,0,...,31,4.041140,1.073946,7.544784,26745.612903,118,406584,152,16,0
10536,VirusShare_622e962ebf6da7ac26aee84873682bce,622e962ebf6da7ac26aee84873682bce,332,224,8450,9,0,89600,55808,0,...,50,2.488268,0.960953,5.020695,257.640000,20,1254,72,16,0
10537,VirusShare_70111012bb702b4cc13c52f7ceb3df91,70111012bb702b4cc13c52f7ceb3df91,332,224,271,6,0,24576,141824,2048,...,15,1.933714,0.000000,5.269173,1210.666667,104,4264,0,0,0


In [4]:
X = df.drop(['Name', 'md5', 'legitimate'], axis = 1).values
y = df['legitimate'].values

In [6]:
from sklearn.feature_selection import SelectFromModel
import sklearn.ensemble as ske
fsel = ske.ExtraTreesClassifier().fit(X, y)
model = SelectFromModel(fsel, prefit=True)
X_new = model.transform(X)
nb_features = X_new.shape[1]
indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]
for f in range(nb_features):
    print("%d. feature %s (%f)" % (f + 1, df.columns[2+indices[f]], fsel.feature_importances_[indices[f]]))
features = []
for f in sorted(np.argsort(fsel.feature_importances_)[::-1][:nb_features]):
    features.append(df.columns[2+f])

1. feature Machine (0.256578)
2. feature MajorSubsystemVersion (0.140497)
3. feature MajorOperatingSystemVersion (0.101406)
4. feature SizeOfOptionalHeader (0.076828)
5. feature Characteristics (0.046599)
6. feature LoadConfigurationSize (0.037570)
7. feature ResourcesMaxEntropy (0.032147)
8. feature DllCharacteristics (0.030352)
9. feature VersionInformationSize (0.026677)
10. feature MajorImageVersion (0.023475)
11. feature ImportsNbDLL (0.023215)


In [8]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.20, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [24]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [25]:
classifier.score(X_test,y_test)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9686907020872866

In [27]:
# Predicting the Test set results
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9813782469457952

In [31]:
pred = classifier.predict(X_test)
p = accuracy_score(y_test, pred)

from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score    
print(classification_report(y_test, pred))

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1417
           1       0.94      0.96      0.95       691

    accuracy                           0.97      2108
   macro avg       0.96      0.97      0.96      2108
weighted avg       0.97      0.97      0.97      2108



In [32]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy')
classifier.fit(X_train, y_train)

#predict the test results
y_pred = classifier.predict(X_test)

#Makeing the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


array([[1401,   16],
       [  22,  669]], dtype=int64)